# Summarize K-Means (K=10) Cluster for full-tfidf

In [2]:
# modify these for your own computer
repo_directory = '/Users/Michael/Documents/GitHub/law-net/'

data_dir = '/Users/Michael/Desktop/network_data/'

import os
import numpy as np
import re
import sys
import matplotlib.pyplot as plt
import glob
import cPickle  as pickle
from collections import OrderedDict
import json


# graph package
import igraph as ig


# stat
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF

from sklearn.metrics import normalized_mutual_info_score as nmi
from sklearn.metrics import adjusted_mutual_info_score as ami
from sklearn.metrics import mutual_info_score as mi
from sklearn.metrics import adjusted_rand_score as ar
from sklearn.metrics import calinski_harabaz_score as ch # (X, labels)
from sklearn.metrics import completeness_score as cs # metric isn't symmetric (labels_true, labels_predicted)
from sklearn.metrics import fowlkes_mallows_score as fm
from sklearn.metrics import homogeneity_completeness_v_measure as hcvm
from sklearn.metrics import homogeneity_score as hs # metric isn't symmetric (labels_true, labels_predicted)
from sklearn.metrics import silhouette_score as ss # (X, labels)
from sklearn.metrics import silhouette_samples as ss2 # (X, labels)
from sklearn.metrics import v_measure_score as vm

import scipy.sparse
import random
import itertools
from itertools import combinations


# our code
sys.path.append(repo_directory + 'code/')
from summarize_clusters import *
from helpful_functions import *

sys.path.append(repo_directory + 'vertex_metrics_experiment/code/')
from bag_of_words import * 

# which network to download data for
network_name = 'scotus' # 'federal', 'ca1', etc


# some sub directories that get used
raw_dir = data_dir + 'raw/'
subnet_dir = data_dir + network_name + '/'
text_dir = subnet_dir + 'textfiles/'
nlp_dir = subnet_dir + 'nlp/'
nlp_sub_dir = nlp_dir + 'bow_tfidf/' #tfidf matrix (and other info, i.e. vocab) computed from bag-of-words matrix
nlp_bow_dir = nlp_dir + 'bow/' #bag-of-words matrix (and other info, i.e. vocab)
nlp_df_sub_dir = nlp_dir + 'bow_tfidf_df/'

# csv location
csv_dir = "C:/Users/Michael/Documents/GitHub/law-net/csv/"
csv_dir_mod = "C:/Users/Michael/Documents/GitHub/law-net/csv/summarize_modularity/"
csv_dir_walk = "C:/Users/Michael/Documents/GitHub/law-net/csv/summarize_walktrap/"
csv_dir_full_tfidf_km10 = "C:/Users/Michael/Documents/GitHub/law-net/csv/summarize_full_tfidf_km10/"

# all the file paths for .txt files
file_paths = glob.glob(text_dir + '*.txt')

# all opinions
all_the_opinions = all_opinions(file_paths)

# clusters directory
clusters_dir = "C:/Users/Michael/Desktop/network_data/raw/scotus/clusters/"

# jupyter notebook settings
%load_ext autoreload
%autoreload 2
%matplotlib inline

## load tf-idf vectors
**tfidf_matrix** = (row_index, column_index): tf_idf value (**CSR FORMAT**)  
**op_id_to_bow_id** = opinion_id (corresponds to row indices)  
**vocab** = all the words in tfidf_matrix (correspond to column indices)

In [3]:
tfidf_matrix, op_id_to_bow_id, vocab = load_tf_idf(nlp_sub_dir)

In [4]:
tfidf_matrix

<27885x567570 sparse matrix of type '<type 'numpy.float64'>'
	with 20817470 stored elements in Compressed Sparse Row format>

In [26]:
clusters = pd.read_csv(csv_dir + 'clusters_full_tfidf.csv')
clusters.head()

,Unnamed: 0,mod,wt,km_10,km_100
0,145658,1,5,8,76
1,89370,3,294,8,76
2,89371,0,35,0,43
3,89372,0,3,8,59
4,89373,0,3,8,11


In [28]:
nlp_tfidf_clusters = clusters['km_10'].tolist()

nlp_clusters = pd.Series(nlp_tfidf_clusters, index=op_id_to_bow_id)
nlp_clusters.head()

145658    8
89370     8
89371     0
89372     8
89373     8
dtype: int64

## get the top 5 biggest clusters

In [30]:
'''
dict_top_n_clusters = dictionary of top K clusters 
                      (key=cluster #, value=opinions in cluster)
                      
biggest_n_clusters = list of top K clusters (int)
'''

dict_top_n_clusters, biggest_n_clusters = get_top_n_clusters(5, 10, nlp_clusters)

cluster 8 : 7168 opinions
cluster 3 : 5027 opinions
cluster 4 : 4084 opinions
cluster 5 : 2853 opinions
cluster 1 : 2048 opinions


# Top K Words of Each Cluster
This function summarizes a set of opinions by returning the words that appear in these opinions with the highest tf-idf scores.

# Top K Words ($\mu_{cluster}$) of Each Cluster
compute the mean tf-idf vector of the cluster, return the top K words from this mean vector

# Top K Words ($\mu_{cluster} - \mu_{complement}$ ) of Each Cluster
compute the mean tf-idf vector of the cluster and also of the complement of the cluster,  
take the difference mu_cluster - mu_complement, return the top K words in this difference

# Most Relevant Opinion of Each Cluster
compute the mean tf-idf vector, return the document in the cluster closet to the mean  

# Cluster 8 Summary (7168 opinions)

In [31]:
%%time

opinion_names = []
opinion_dates = []
opinion_links = []

for i in dict_top_n_clusters[8]:
    try:
        with open(clusters_dir + i + ".json") as data_file:
            data = json.load(data_file)
    except IOError:
        pass
        #name, date, link = case_info2(i)
        #opinion_names.append(name)
        #opinion_dates.append(date)
        #opinion_links.append(link)
        
    name = data['case_name'].encode('utf-8')
    date = data['date_filed'].encode('utf-8')
    link = 'https://www.courtlistener.com' + data['absolute_url'].encode('utf-8')
    
    opinion_names.append(name)
    opinion_dates.append(date)
    opinion_links.append(link)
    
cluster_info = pd.DataFrame()
cluster_info['names'] = opinion_names
cluster_info['dates'] = opinion_dates
cluster_info['url'] = opinion_links

cluster_info.to_csv(csv_dir_full_tfidf_km10 + "cluster_8.csv")

print cluster_info.shape

(7168, 3)
Wall time: 3.65 s


In [32]:
cluster_info = pd.read_csv(csv_dir_full_tfidf_km10 + 'cluster_8.csv')
cluster_info.head()

,Unnamed: 0,names,dates,url
0,0,DaimlerChrysler Corp. v. Cuno,2006-05-15,https://www.courtlistener.com/opinion/145658/d...
1,1,Garfielde v. United States,1876-05-18,https://www.courtlistener.com/opinion/89370/ga...
2,2,Barkley v. Levee Commissioners,1876-12-18,https://www.courtlistener.com/opinion/89372/ba...
3,3,Broughton v. Pensacola,1876-12-18,https://www.courtlistener.com/opinion/89373/br...
4,4,Sawin v. Kenny,1876-12-18,https://www.courtlistener.com/opinion/89377/sa...


In [33]:
%%time

k=1000 # number of words to get

top_words = top_k_words(dict_top_n_clusters[8], k, tfidf_matrix, op_id_to_bow_id, vocab)
top_words_from_mean = top_k_words_from_mean_vector(dict_top_n_clusters[8], k, tfidf_matrix, op_id_to_bow_id, vocab)
top_words_from_diff = top_k_words_from_difference(dict_top_n_clusters[8], all_the_opinions, k, tfidf_matrix, op_id_to_bow_id, vocab)
most_relev_op = document_closest_to_mean(dict_top_n_clusters[8], tfidf_matrix, op_id_to_bow_id)

top_words = [x.encode('utf-8') for x in top_words]
top_words_from_mean = [x.encode('utf-8') for x in top_words_from_mean]
top_words_from_diff = [x.encode('utf-8') for x in top_words_from_diff]

#print '\x1b[1;31m' + "Top K Words:" + '\x1b[0m', top_words
#print '\x1b[1;31m' + "Top K Words (Mu_Cluster):" + '\x1b[0m', top_words_from_mean
#print '\x1b[1;31m' + "Top K Words (Mu_Cluster - Mu_Complement):" + '\x1b[0m', top_words_from_diff
#print '\x1b[1;31m' + "Most Relevent Opinion:" + '\x1b[0m', most_relev_op

cluster_summary = pd.DataFrame()
cluster_summary['top_words'] = top_words
cluster_summary['top_words_from_mean'] = top_words_from_mean
cluster_summary['top_words_from_diff'] = top_words_from_diff
cluster_summary['most_relev_op'] = most_relev_op

cluster_summary.to_csv(csv_dir_full_tfidf_km10 + "cluster_8_summary.csv")

print cluster_summary.shape

(1000, 4)
Wall time: 1min 9s


In [34]:
cluster_summary = pd.read_csv(csv_dir_full_tfidf_km10 + 'cluster_8_summary.csv')
cluster_summary.head()

,Unnamed: 0,top_words,top_words_from_mean,top_words_from_diff,most_relev_op
0,0,chicori,court,patent,96819
1,1,squier,state,suit,96819
2,2,baal,v,properti,96819
3,3,arbitr,case,certif,96819
4,4,wenzel,act,claimant,96819


# Cluster 3 Summary (5027 opinions)

In [35]:
%%time

opinion_names = []
opinion_dates = []
opinion_links = []

for i in dict_top_n_clusters[3]:
    try:
        with open(clusters_dir + i + ".json") as data_file:
            data = json.load(data_file)
    except IOError:
        pass
        #name, date, link = case_info2(i)
        #opinion_names.append(name)
        #opinion_dates.append(date)
        #opinion_links.append(link)
        
    name = data['case_name'].encode('utf-8')
    date = data['date_filed'].encode('utf-8')
    link = 'https://www.courtlistener.com' + data['absolute_url'].encode('utf-8')
    
    opinion_names.append(name)
    opinion_dates.append(date)
    opinion_links.append(link)
    
cluster_info = pd.DataFrame()
cluster_info['names'] = opinion_names
cluster_info['dates'] = opinion_dates
cluster_info['url'] = opinion_links

cluster_info.to_csv(csv_dir_full_tfidf_km10 + "cluster_3.csv")

print cluster_info.shape

(5027, 3)
Wall time: 2.67 s


In [36]:
cluster_info = pd.read_csv(csv_dir_full_tfidf_km10 + 'cluster_3.csv')
cluster_info.head()

,Unnamed: 0,names,dates,url
0,0,Hawkins v. Blake,1883-05-07,https://www.courtlistener.com/opinion/90875/ha...
1,1,Maynard v. Hill,1888-03-19,https://www.courtlistener.com/opinion/92187/ma...
2,2,"James Rowland, Former Director, California Dep...",1993-01-12,https://www.courtlistener.com/opinion/2764189/...
3,3,Moore v. Mississippi,1875-04-19,https://www.courtlistener.com/opinion/89041/mo...
4,4,Guaranty Trust Co. v. United States,1938-04-25,https://www.courtlistener.com/opinion/103016/g...


In [37]:
%%time

k=1000 # number of words to get

top_words = top_k_words(dict_top_n_clusters[3], k, tfidf_matrix, op_id_to_bow_id, vocab)
top_words_from_mean = top_k_words_from_mean_vector(dict_top_n_clusters[3], k, tfidf_matrix, op_id_to_bow_id, vocab)
top_words_from_diff = top_k_words_from_difference(dict_top_n_clusters[3], all_the_opinions, k, tfidf_matrix, op_id_to_bow_id, vocab)
most_relev_op = document_closest_to_mean(dict_top_n_clusters[3], tfidf_matrix, op_id_to_bow_id)

top_words = [x.encode('utf-8') for x in top_words]
top_words_from_mean = [x.encode('utf-8') for x in top_words_from_mean]
top_words_from_diff = [x.encode('utf-8') for x in top_words_from_diff]

#print '\x1b[1;31m' + "Top K Words:" + '\x1b[0m', top_words
#print '\x1b[1;31m' + "Top K Words (Mu_Cluster):" + '\x1b[0m', top_words_from_mean
#print '\x1b[1;31m' + "Top K Words (Mu_Cluster - Mu_Complement):" + '\x1b[0m', top_words_from_diff
#print '\x1b[1;31m' + "Most Relevent Opinion:" + '\x1b[0m', most_relev_op

cluster_summary = pd.DataFrame()
cluster_summary['top_words'] = top_words
cluster_summary['top_words_from_mean'] = top_words_from_mean
cluster_summary['top_words_from_diff'] = top_words_from_diff
cluster_summary['most_relev_op'] = most_relev_op

cluster_summary.to_csv(csv_dir_full_tfidf_km10 + "cluster_3_summary.csv")

print cluster_summary.shape

(1000, 4)
Wall time: 49.1 s


In [38]:
cluster_summary = pd.read_csv(csv_dir_full_tfidf_km10 + 'cluster_3_summary.csv')
cluster_summary.head()

,Unnamed: 0,top_words,top_words_from_mean,top_words_from_diff,most_relev_op
0,0,shawhan,court,arbitr,95314
1,1,corsica,state,commiss,95314
2,2,arbitr,v,petition,95314
3,3,vetterlein,case,act,95314
4,4,metsker,act,liquor,95314


# Cluster 4 Summary (4084 opinions)

In [39]:
%%time

opinion_names = []
opinion_dates = []
opinion_links = []

for i in dict_top_n_clusters[4]:
    try:
        with open(clusters_dir + i + ".json") as data_file:
            data = json.load(data_file)
    except IOError:
        pass
        #name, date, link = case_info2(i)
        #opinion_names.append(name)
        #opinion_dates.append(date)
        #opinion_links.append(link)
        
    name = data['case_name'].encode('utf-8')
    date = data['date_filed'].encode('utf-8')
    link = 'https://www.courtlistener.com' + data['absolute_url'].encode('utf-8')
    
    opinion_names.append(name)
    opinion_dates.append(date)
    opinion_links.append(link)
    
cluster_info = pd.DataFrame()
cluster_info['names'] = opinion_names
cluster_info['dates'] = opinion_dates
cluster_info['url'] = opinion_links

cluster_info.to_csv(csv_dir_full_tfidf_km10 + "cluster_4.csv")

print cluster_info.shape

(4084, 3)
Wall time: 2.32 s


In [40]:
cluster_info = pd.read_csv(csv_dir_full_tfidf_km10 + 'cluster_4.csv')
cluster_info.head()

,Unnamed: 0,names,dates,url
0,0,Dalton v. Jennings,1876-12-18,https://www.courtlistener.com/opinion/89374/da...
1,1,"Chicago, M. & St. PR Co. v. United States",1888-05-14,https://www.courtlistener.com/opinion/92267/ch...
2,2,Grover & Baker Sewing MacHine Co. v. Radcliffe,1890-12-08,https://www.courtlistener.com/opinion/92880/gr...
3,3,The Burlington,1890-12-15,https://www.courtlistener.com/opinion/92889/th...
4,4,McAllister v. United States,1891-05-25,https://www.courtlistener.com/opinion/93137/mc...


In [41]:
%%time

k=1000 # number of words to get

top_words = top_k_words(dict_top_n_clusters[4], k, tfidf_matrix, op_id_to_bow_id, vocab)
top_words_from_mean = top_k_words_from_mean_vector(dict_top_n_clusters[4], k, tfidf_matrix, op_id_to_bow_id, vocab)
top_words_from_diff = top_k_words_from_difference(dict_top_n_clusters[4], all_the_opinions, k, tfidf_matrix, op_id_to_bow_id, vocab)
most_relev_op = document_closest_to_mean(dict_top_n_clusters[4], tfidf_matrix, op_id_to_bow_id)

top_words = [x.encode('utf-8') for x in top_words]
top_words_from_mean = [x.encode('utf-8') for x in top_words_from_mean]
top_words_from_diff = [x.encode('utf-8') for x in top_words_from_diff]

#print '\x1b[1;31m' + "Top K Words:" + '\x1b[0m', top_words
#print '\x1b[1;31m' + "Top K Words (Mu_Cluster):" + '\x1b[0m', top_words_from_mean
#print '\x1b[1;31m' + "Top K Words (Mu_Cluster - Mu_Complement):" + '\x1b[0m', top_words_from_diff
#print '\x1b[1;31m' + "Most Relevent Opinion:" + '\x1b[0m', most_relev_op

cluster_summary = pd.DataFrame()
cluster_summary['top_words'] = top_words
cluster_summary['top_words_from_mean'] = top_words_from_mean
cluster_summary['top_words_from_diff'] = top_words_from_diff
cluster_summary['most_relev_op'] = most_relev_op

cluster_summary.to_csv(csv_dir_full_tfidf_km10 + "cluster_4_summary.csv")

print cluster_summary.shape

(1000, 4)
Wall time: 39 s


In [42]:
cluster_summary = pd.read_csv(csv_dir_full_tfidf_km10 + 'cluster_4_summary.csv')
cluster_summary.head()

,Unnamed: 0,top_words,top_words_from_mean,top_words_from_diff,most_relev_op
0,0,skillern,court,vessel,104135
1,1,pdf,state,tax,104135
2,2,martindal,v,water,104135
3,3,colburn,case,bond,104135
4,4,artwar,act,sentenc,104135


# Cluster 5 Summary (2853 opinions)

In [43]:
%%time

opinion_names = []
opinion_dates = []
opinion_links = []

for i in dict_top_n_clusters[5]:
    try:
        with open(clusters_dir + i + ".json") as data_file:
            data = json.load(data_file)
    except IOError:
        pass
        #name, date, link = case_info2(i)
        #opinion_names.append(name)
        #opinion_dates.append(date)
        #opinion_links.append(link)
        
    name = data['case_name'].encode('utf-8')
    date = data['date_filed'].encode('utf-8')
    link = 'https://www.courtlistener.com' + data['absolute_url'].encode('utf-8')
    
    opinion_names.append(name)
    opinion_dates.append(date)
    opinion_links.append(link)
    
cluster_info = pd.DataFrame()
cluster_info['names'] = opinion_names
cluster_info['dates'] = opinion_dates
cluster_info['url'] = opinion_links

cluster_info.to_csv(csv_dir_full_tfidf_km10 + "cluster_5.csv")

print cluster_info.shape

(2853, 3)
Wall time: 1.34 s


In [44]:
cluster_info = pd.read_csv(csv_dir_full_tfidf_km10 + 'cluster_5.csv')
cluster_info.head()

,Unnamed: 0,names,dates,url
0,0,Oklahoma Ex Rel. Phillips v. Guy F. Atkinson Co.,1941-06-02,https://www.courtlistener.com/opinion/103544/o...
1,1,Gt. No. Ry. v. Merchants Elev. Co.,1922-05-29,https://www.courtlistener.com/opinion/100011/g...
2,2,Schaefer v. Werling,1903-02-23,https://www.courtlistener.com/opinion/95799/sc...
3,3,Wilmot v. Mudge,1881-04-25,https://www.courtlistener.com/opinion/90313/wi...
4,4,Carothers v. Mayer,1896-11-30,https://www.courtlistener.com/opinion/94547/ca...


In [45]:
%%time

k=1000 # number of words to get

top_words = top_k_words(dict_top_n_clusters[5], k, tfidf_matrix, op_id_to_bow_id, vocab)
top_words_from_mean = top_k_words_from_mean_vector(dict_top_n_clusters[5], k, tfidf_matrix, op_id_to_bow_id, vocab)
top_words_from_diff = top_k_words_from_difference(dict_top_n_clusters[5], all_the_opinions, k, tfidf_matrix, op_id_to_bow_id, vocab)
most_relev_op = document_closest_to_mean(dict_top_n_clusters[5], tfidf_matrix, op_id_to_bow_id)

top_words = [x.encode('utf-8') for x in top_words]
top_words_from_mean = [x.encode('utf-8') for x in top_words_from_mean]
top_words_from_diff = [x.encode('utf-8') for x in top_words_from_diff]

#print '\x1b[1;31m' + "Top K Words:" + '\x1b[0m', top_words
#print '\x1b[1;31m' + "Top K Words (Mu_Cluster):" + '\x1b[0m', top_words_from_mean
#print '\x1b[1;31m' + "Top K Words (Mu_Cluster - Mu_Complement):" + '\x1b[0m', top_words_from_diff
#print '\x1b[1;31m' + "Most Relevent Opinion:" + '\x1b[0m', most_relev_op

cluster_summary = pd.DataFrame()
cluster_summary['top_words'] = top_words
cluster_summary['top_words_from_mean'] = top_words_from_mean
cluster_summary['top_words_from_diff'] = top_words_from_diff
cluster_summary['most_relev_op'] = most_relev_op

cluster_summary.to_csv(csv_dir_full_tfidf_km10 + "cluster_5_summary.csv")

print cluster_summary.shape

(1000, 4)
Wall time: 27.1 s


In [46]:
cluster_summary = pd.read_csv(csv_dir_full_tfidf_km10 + 'cluster_5_summary.csv')
cluster_summary.head()

,Unnamed: 0,top_words,top_words_from_mean,top_words_from_diff,most_relev_op
0,0,graeff,court,land,102112
1,1,millner,state,decre,102112
2,2,bicknel,v,court,102112
3,3,ashcraft,case,case,102112
4,4,kountz,act,petition,102112


# Cluster 1 Summary (2048 opinions)

In [47]:
%%time

opinion_names = []
opinion_dates = []
opinion_links = []

for i in dict_top_n_clusters[1]:
    try:
        with open(clusters_dir + i + ".json") as data_file:
            data = json.load(data_file)
    except IOError:
        pass
        #name, date, link = case_info2(i)
        #opinion_names.append(name)
        #opinion_dates.append(date)
        #opinion_links.append(link)
        
    name = data['case_name'].encode('utf-8')
    date = data['date_filed'].encode('utf-8')
    link = 'https://www.courtlistener.com' + data['absolute_url'].encode('utf-8')
    
    opinion_names.append(name)
    opinion_dates.append(date)
    opinion_links.append(link)
    
cluster_info = pd.DataFrame()
cluster_info['names'] = opinion_names
cluster_info['dates'] = opinion_dates
cluster_info['url'] = opinion_links

cluster_info.to_csv(csv_dir_full_tfidf_km10 + "cluster_1.csv")

print cluster_info.shape

(2048, 3)
Wall time: 974 ms


In [49]:
cluster_info = pd.read_csv(csv_dir_full_tfidf_km10 + 'cluster_1.csv')
cluster_info.head()

,Unnamed: 0,names,dates,url
0,0,Indianapolis & St. Louis R. Co. v. Horst,1876-12-18,https://www.courtlistener.com/opinion/89378/in...
1,1,"The"" Atlas""",1876-11-27,https://www.courtlistener.com/opinion/89379/th...
2,2,Wilson v. Daniel,1798-08-17,https://www.courtlistener.com/opinion/2224796/...
3,3,Unio Pac. R. Co. v. United States,1941-10-13,https://www.courtlistener.com/opinion/103541/u...
4,4,United States v. AS Kreider Co.,1941-05-26,https://www.courtlistener.com/opinion/103540/u...


In [50]:
%%time

k=1000 # number of words to get

top_words = top_k_words(dict_top_n_clusters[1], k, tfidf_matrix, op_id_to_bow_id, vocab)
top_words_from_mean = top_k_words_from_mean_vector(dict_top_n_clusters[1], k, tfidf_matrix, op_id_to_bow_id, vocab)
top_words_from_diff = top_k_words_from_difference(dict_top_n_clusters[1], all_the_opinions, k, tfidf_matrix, op_id_to_bow_id, vocab)
most_relev_op = document_closest_to_mean(dict_top_n_clusters[1], tfidf_matrix, op_id_to_bow_id)

top_words = [x.encode('utf-8') for x in top_words]
top_words_from_mean = [x.encode('utf-8') for x in top_words_from_mean]
top_words_from_diff = [x.encode('utf-8') for x in top_words_from_diff]

#print '\x1b[1;31m' + "Top K Words:" + '\x1b[0m', top_words
#print '\x1b[1;31m' + "Top K Words (Mu_Cluster):" + '\x1b[0m', top_words_from_mean
#print '\x1b[1;31m' + "Top K Words (Mu_Cluster - Mu_Complement):" + '\x1b[0m', top_words_from_diff
#print '\x1b[1;31m' + "Most Relevent Opinion:" + '\x1b[0m', most_relev_op

cluster_summary = pd.DataFrame()
cluster_summary['top_words'] = top_words
cluster_summary['top_words_from_mean'] = top_words_from_mean
cluster_summary['top_words_from_diff'] = top_words_from_diff
cluster_summary['most_relev_op'] = most_relev_op

cluster_summary.to_csv(csv_dir_full_tfidf_km10 + "cluster_1_summary.csv")

print cluster_summary.shape

(1000, 4)
Wall time: 23.1 s


In [51]:
cluster_summary = pd.read_csv(csv_dir_full_tfidf_km10 + 'cluster_1_summary.csv')
cluster_summary.head()

,Unnamed: 0,top_words,top_words_from_mean,top_words_from_diff,most_relev_op
0,0,scophoni,court,wit,97121
1,1,stumpf,state,court,97121
2,2,meserv,v,defend,97121
3,3,lick,case,bankrupt,97121
4,4,tray,act,clerk,97121
